In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Example 3D CNN model (replace with your actual model)
class Sample3DCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv3d(64, 128, 3, padding=1)
        self.conv5 = nn.Conv3d(128, 256, 3, padding=1)
        self.pool = nn.AdaptiveAvgPool3d(1)  # For GAP

    def forward(self, x):
        features = []
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        features.append(x)  # fifth-last (if conv5 is last)
        x = F.relu(self.conv3(x))
        features.append(x)  # third-last
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        features.append(x)  # last
        return features

# Initialize model
model = Sample3DCNN()
model.eval()

# Simulated 3D input volumes per region (replace with real data)
regions = {
    "Tibia": torch.randn(1, 1, 64, 64, 64),
    "Femur": torch.randn(1, 1, 64, 64, 64),
    "Background": torch.randn(1, 1, 64, 64, 64)
}

# Store output feature vectors
region_features = {}

with torch.no_grad():
    for region_name, volume in regions.items():
        feature_maps = model(volume)

        # Apply Global Average Pooling to each selected feature map
        pooled_vectors = [F.adaptive_avg_pool3d(fm, 1).flatten(1) for fm in feature_maps]

        # Optional: concatenate all pooled vectors
        feature_vector = torch.cat(pooled_vectors, dim=1)

        region_features[region_name] = feature_vector

# Example output
for region, vec in region_features.items():
    print(f"{region} feature vector shape: {vec.shape}")


Tibia feature vector shape: torch.Size([1, 352])
Femur feature vector shape: torch.Size([1, 352])
Background feature vector shape: torch.Size([1, 352])
